In [1]:
import numpy as np
import pandas as pd

from warnings import simplefilter
simplefilter('ignore')

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 20)

In [2]:
from performStockAnalysis.main import select_emiten_to_model, develop_models_for_selected_emiten, forecast_using_the_developed_models

In [3]:
# selected_emiten = select_emiten_to_model()

In [4]:
develop_models_for_selected_emiten(['BBCA', 'BMRI', 'GPSO'], 'median_gain', [10, 15])

2025-10-31 06:29:49 - INFO - ===== Starting Model Development for 3 Selected Emitens =====
2025-10-31 06:29:49 - INFO - Processing Emiten: BBCA (1/3)
2025-10-31 06:29:49 - INFO - ----- Starting Data Preparation Pipeline for Emiten BBCA -----
2025-10-31 06:29:49 - INFO - Downloading the stock data
2025-10-31 06:29:50 - INFO - Generating technical indicators as features
2025-10-31 06:29:50 - INFO - Processing the data into a format that is compatible for generating the stock indicators
2025-10-31 06:29:50 - INFO - Generating stock indicators of price trends
2025-10-31 06:29:50 - INFO - Generating stock indicators of price channels
2025-10-31 06:29:52 - INFO - Generating stock indicators of oscillators
2025-10-31 06:29:52 - INFO - Generating stock indicators of volume based
2025-10-31 06:29:54 - INFO - Generating stock indicators of price transformations
2025-10-31 06:29:54 - INFO - Successfully generated all stock indicators
2025-10-31 06:29:54 - INFO - Acquired 52 features for the stock

In [5]:
performance_10dd = pd.read_csv('database/modelPerformances/medianGain/10dd-20251031.csv') \
                        .dropna() \
                        .reset_index(drop=True) \
                        .sort_values('Threshold', ascending=False)

performance_15dd = pd.read_csv('database/modelPerformances/medianGain/15dd-20251031.csv') \
                        .dropna() \
                        .reset_index(drop=True) \
                        .sort_values('Threshold', ascending=False)

In [6]:
performance_10dd.describe()

,Train - Accuracy,Train - Precision High Gain,Train - Precision Low Gain,Train - Recall High Gain,Train - Recall Low Gain,Train - Gini,Test - Accuracy,Test - Precision High Gain,Test - Precision Low Gain,Test - Recall High Gain,Test - Recall Low Gain,Test - Gini,Threshold
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,0.971748,0.975680,0.975680,0.749654,0.993789,0.969390,0.895833,0.913683,0.913683,0.632862,0.961504,0.869896,3.682890
std,0.016030,0.012695,0.012695,0.132436,0.004558,0.027968,0.028701,0.031654,0.031654,0.150454,0.015751,0.071836,2.513249
min,0.960199,0.966916,0.966916,0.655556,0.990164,0.942720,0.875000,0.882353,0.882353,0.473684,0.943820,0.822283,1.869311
25%,0.962597,0.968401,0.968401,0.673932,0.991231,0.954837,0.879464,0.897698,0.897698,0.562929,0.955243,0.828582,2.248473
50%,0.964994,0.969887,0.969887,0.692308,0.992298,0.966953,0.883929,0.913043,0.913043,0.652174,0.966667,0.834880,2.627635
75%,0.977522,0.980063,0.980063,0.796703,0.995602,0.982725,0.906250,0.929348,0.929348,0.712451,0.970346,0.893703,4.589680
max,0.990050,0.990239,0.990239,0.901099,0.998906,0.998497,0.928571,0.945652,0.945652,0.772727,0.974026,0.952525,6.551724


In [7]:
performance_15dd.describe()

,Train - Accuracy,Train - Precision High Gain,Train - Precision Low Gain,Train - Recall High Gain,Train - Recall Low Gain,Train - Gini,Test - Accuracy,Test - Precision High Gain,Test - Precision Low Gain,Test - Recall High Gain,Test - Recall Low Gain,Test - Gini,Threshold
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,0.982249,0.983767,0.983767,0.824381,0.997098,0.988116,0.887401,0.898293,0.898293,0.622664,0.964749,0.852160,4.522371
std,0.012500,0.010946,0.010946,0.120098,0.002513,0.010263,0.052408,0.045731,0.045731,0.145943,0.022242,0.066052,3.345121
min,0.973134,0.975454,0.975454,0.735632,0.995643,0.982158,0.857143,0.863158,0.863158,0.518519,0.942529,0.778851,2.298400
25%,0.975124,0.977566,0.977566,0.756051,0.995647,0.982191,0.857143,0.872439,0.872439,0.539259,0.953617,0.824719,2.598872
50%,0.977114,0.979679,0.979679,0.776471,0.995652,0.982225,0.857143,0.881720,0.881720,0.560000,0.964706,0.870588,2.899344
75%,0.986807,0.987924,0.987924,0.868755,0.997826,0.991096,0.902530,0.915860,0.915860,0.674737,0.975859,0.888814,5.634357
max,0.996499,0.996169,0.996169,0.961039,1.000000,0.999967,0.947917,0.950000,0.950000,0.789474,0.987013,0.907040,8.369369


In [8]:
forecast_using_the_developed_models(10, 'medianGain', '20251031', 0.5)

2025-10-31 06:32:13 - INFO - ===== Starting the Process of 10 Days Forecasting on Mediangain Label Type =====
2025-10-31 06:32:13 - INFO - Loading the model's performance data from database/modelPerformances/medianGain/10dd-20251031.csv
2025-10-31 06:32:13 - INFO - Select stocks with a Gini performance on testing data that is greater than 0.5
2025-10-31 06:32:13 - INFO - Selected a total of 3 stocks that exceed the minimum model's performance
2025-10-31 06:32:13 - INFO - Loading stock's technical indicators as features
2025-10-31 06:32:13 - INFO - Prepare all stock data to be used for forecasting
2025-10-31 06:32:13 - INFO - ----- Starting Data Preparation Pipeline for Emiten BMRI -----
2025-10-31 06:32:13 - INFO - Downloading stock data
2025-10-31 06:32:14 - INFO - Generating technical indicators as features
2025-10-31 06:32:14 - INFO - Processing the data into a format that is compatible for generating the stock indicators
2025-10-31 06:32:14 - INFO - Generating stock indicators of p

In [9]:
forecast_using_the_developed_models(15, 'median_gain', '20251031', 0.5)

2025-10-31 06:32:27 - INFO - ===== Starting the Process of 15 Days Forecasting on Mediangain Label Type =====
2025-10-31 06:32:27 - INFO - Loading the model's performance data from database/modelPerformances/medianGain/15dd-20251031.csv
2025-10-31 06:32:27 - INFO - Select stocks with a Gini performance on testing data that is greater than 0.5
2025-10-31 06:32:27 - INFO - Selected a total of 3 stocks that exceed the minimum model's performance
2025-10-31 06:32:27 - INFO - Loading stock's technical indicators as features
2025-10-31 06:32:27 - INFO - Prepare all stock data to be used for forecasting
2025-10-31 06:32:27 - INFO - ----- Starting Data Preparation Pipeline for Emiten GPSO -----
2025-10-31 06:32:27 - INFO - Downloading stock data
2025-10-31 06:32:27 - INFO - Generating technical indicators as features
2025-10-31 06:32:27 - INFO - Processing the data into a format that is compatible for generating the stock indicators
2025-10-31 06:32:27 - INFO - Generating stock indicators of p

In [10]:
forecast_10_dd = pd.read_csv('database/forecastedStocks/medianGain/forecast-10dd-20251031.csv')
forecast_15_dd = pd.read_csv('database/forecastedStocks/medianGain/forecast-15dd-20251031.csv')

In [11]:
merged_forecast = pd.merge(performance_10dd, forecast_10_dd, on='Kode').sort_values('Forecast High Gain 10dd', ascending=False)

In [13]:
merged_forecast

,Kode,Train - Accuracy,Train - Precision High Gain,Train - Precision Low Gain,Train - Recall High Gain,Train - Recall Low Gain,Train - Gini,Test - Accuracy,Test - Precision High Gain,Test - Precision Low Gain,Test - Recall High Gain,Test - Recall Low Gain,Test - Gini,Threshold,Date,Forecast High Gain 10dd
0,GPSO,0.964994,0.969887,0.969887,0.692308,0.992298,0.966953,0.875000,0.882353,0.882353,0.473684,0.974026,0.822283,6.551724,2025-10-30,0.248837
2,BBCA,0.960199,0.966916,0.966916,0.655556,0.990164,0.942720,0.883929,0.913043,0.913043,0.652174,0.943820,0.834880,1.869311,2025-10-30,0.003356
1,BMRI,0.990050,0.990239,0.990239,0.901099,0.998906,0.998497,0.928571,0.945652,0.945652,0.772727,0.966667,0.952525,2.627635,2025-10-30,0.000217
